In [1]:
%load_ext autoreload
%autoreload 2

In [82]:
import csv
import numpy
# You may or may not want to use this package, or others like it
# this is just a starting point for you
from sklearn.linear_model import LinearRegression

import pandas as pd
import os

In [83]:
# Read the player database into an array of dictionaries
players = []
with open('playerDB.csv', mode='r') as player_csv:
    player_reader = csv.DictReader(player_csv)
    line_count = 0
    for row in player_reader:
        players.append(dict(row))

# Read the draft database into an array of dictionaries
draftPicks = []
with open('draftDB.csv', mode='r') as draft_csv:
    draft_reader = csv.DictReader(draft_csv)
    line_count = 0
    for row in draft_reader:
        draftPicks.append(dict(row))


In [93]:
draft_df.head()

,yearDraft,numberPickOverall,numberRound,numberRoundPick,namePlayer,slugTeam,nameOrganizationFrom,typeOrganizationFrom,idPlayer,idTeam,...,NBAID,NBABirthDate,ESPNName,ESPNLink,ESPNID,ESPNBirthDate,SpotracName,SpotracLink,SpotracID,urlID
0,2015,1,1,1,Karl-Anthony Towns,MIN,Kentucky,College/University,1626157,1610612750,...,1626157.0,11/15/1995,Karl-Anthony Towns,http://www.espn.com/nba/player/_/id/3136195/ka...,3136195.0,11/15/1995,Karl-Anthony Towns,https://www.spotrac.com/redirect/player/17829/,17829.0,townska01
1,2015,2,1,2,D'Angelo Russell,LAL,Ohio State,College/University,1626156,1610612747,...,1626156.0,2/23/1996,D'Angelo Russell,http://www.espn.com/nba/player/_/id/3136776/da...,3136776.0,2/23/1996,D'Angelo Russell,https://www.spotrac.com/redirect/player/17830/,17830.0,russeda01
2,2015,3,1,3,Jahlil Okafor,PHI,Duke,College/University,1626143,1610612755,...,1626143.0,12/15/1995,Jahlil Okafor,http://www.espn.com/nba/player/_/id/3135048/ja...,3135048.0,12/15/1995,Jahlil Okafor,https://www.spotrac.com/redirect/player/17831/,17831.0,okafoja01
3,2015,4,1,4,Kristaps Porzingis,NYK,CB Sevilla,Other Team/Club,204001,1610612752,...,204001.0,8/2/1995,Kristaps Porzingis,http://www.espn.com/nba/player/_/id/3102531/kr...,3102531.0,8/2/1995,Kristaps Porzingis,https://www.spotrac.com/redirect/player/17832/,17832.0,porzikr01
4,2015,5,1,5,Mario Hezonja,ORL,FC Barcelona Basquet,Other Team/Club,1626209,1610612753,...,1626209.0,2/25/1995,Mario Hezonja,http://www.espn.com/nba/player/_/id/2995706/ma...,2995706.0,2/25/1995,Mario Hezonja,https://www.spotrac.com/redirect/player/17833/,17833.0,hezonma01


In [121]:
players_df = pd.read_csv('playerDB.csv')
draft_df = pd.read_csv('draftDB.csv')

link_df = pd.read_csv('NBA_Player_IDs.csv',  encoding= 'unicode_escape')
link_df = link_df[pd.notna(link_df['NBAID'])]
link_df.loc[:,'NBAID'] = link_df.loc[:,'NBAID'].astype(int)

draft_df = draft_df[draft_df['numberRound']<=2]
draft_df = draft_df[draft_df['yearDraft']>=1979]
draft_df = draft_df[draft_df['yearDraft']<=2015]

#draft_df.loc[:,'urlID'] = draft_df['basketball_reference_url'].apply(lambda url: os.path.splitext(os.path.basename(str(url)))[0])
players_df.loc[:,'year'] = players_df['Season'].apply(lambda season: int(season.split('-')[0]))

curr_miss = draft_df[draft_df.basketball_reference_url.isna()]
print(f'Missing {len(curr_miss)} links between player and draft')

draft_df = draft_df.merge(link_df, how='left', left_on='idPlayer', right_on='NBAID')
draft_df.loc[:,'urlID'] = draft_df.loc[:,'BBRefID'] 

curr_miss = draft_df[draft_df.urlID.isna()]
print(f'Missing {len(curr_miss)} links between player and draft')

combined_df = players_df.merge(draft_df, how='inner', on='urlID')

num_players_yrs = len(combined_df)
num_players = len(combined_df.urlID.unique())

print(f'{num_players} distinct players with {num_players_yrs} player-seasons of data')

only_4_years = combined_df[(combined_df['year']-combined_df['yearDraft'])<=3]
tot_ws_rookie = only_4_years.groupby('urlID')['WS'].sum().rename('tot_WS')
only_4_tot_ws = only_4_years.merge(tot_ws_rookie,left_on='urlID',right_index=True)
avg_rookie_ws = only_4_tot_ws.groupby('numberPickOverall')['tot_WS'].mean()
#avg_rookie_ws.to_csv('avg_rookie_winshare.csv')

Missing 684 links between player and draft
Missing 297 links between player and draft
1731 distinct players with 12087 player-seasons of data


In [122]:
def trade_eval(give_picks, receive_picks):
    give_val = avg_rookie_ws.loc[give_picks].sum()
    receive_val = avg_rookie_ws[receive_picks].sum()
    
    success = receive_val > give_val
    if success:
        print("\nTrade result: Success! This trade receives more value than it gives away.\n")
        # Print additional metrics/reasoning here
    else:
        print("\nTrade result: Don't do it! This trade gives away more value than it receives.\n")
        # Print additional metrics/reasoning here
        
    print(f'Give val: {give_val}')
    print(f'Receive val: {receive_val}')
    

In [123]:
trade_eval([1,15],[2,18])


Trade result: Don't do it! This trade gives away more value than it receives.

Give val: 33.88484673221515
Receive val: 27.86410071942446


In [124]:
avg_rookie_ws

numberPickOverall
1     26.261538
2     17.274101
3     21.975524
4     15.984932
5     16.865035
6     12.782014
7     13.543537
8     10.229787
9     15.028966
10    13.023776
11    11.972535
12    10.469118
13    11.127465
14     8.851852
15     7.623308
16     8.172180
17     7.436641
18    10.590000
19     7.386466
20     7.857600
21     9.376800
22     8.240000
23     8.237815
24     8.603361
25     5.372951
26     7.513462
27     7.210909
28     6.658586
29     6.147525
30     8.603409
31     6.376471
32     4.347761
33     5.298947
34     4.775000
35     6.335897
36     3.253247
37     6.470588
38     5.304348
39     4.303947
40     5.298551
41     5.242029
42     4.614286
43     5.021839
44     3.656000
45     5.831884
46     5.878788
47     6.392063
48     5.800000
49     4.184091
50     4.310000
51     3.863636
52     2.768889
53     0.861538
54     3.406250
55     3.085185
56     3.281481
57     1.280000
58     2.873333
59    -0.200000
60     7.126667
Name: tot_WS, dtype: f

In [116]:
avg_rookie_ws

numberPickOverall
1     26.261538
2     17.274101
3     21.975524
4     15.984932
5     16.865035
6     12.782014
7     13.543537
8     10.229787
9     15.028966
10    13.023776
11    11.972535
12    10.469118
13    11.127465
14     8.851852
15     7.623308
16     8.172180
17     7.436641
18    10.590000
19     7.386466
20     7.857600
21     9.376800
22     8.240000
23     8.237815
24     8.603361
25     5.372951
26     7.513462
27     7.210909
28     6.658586
29     6.147525
30     8.603409
31     6.376471
32     4.347761
33     5.298947
34     4.775000
35     6.335897
36     3.253247
37     6.470588
38     5.304348
39     4.303947
40     5.298551
41     5.242029
42     4.614286
43     5.021839
44     3.656000
45     5.831884
46     5.878788
47     6.392063
48     5.800000
49     4.184091
50     4.310000
51     3.863636
52     2.768889
53     0.861538
54     3.406250
55     3.085185
56     3.281481
57     1.280000
58     2.873333
59    -0.200000
60     7.126667
Name: tot_WS, dtype: f


Trade result: Don't do it! This trade gives away more value than it receives.

Give val: 33.88484673221515
Receive val: 27.86410071942446


In [66]:
302/2268

0.13315696649029982

In [98]:
players_df[players_df['Player']=='Olivier Hanlan']

,Season,Rk,Player,urlID,Pos,Age,Tm,G,MP,PER,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,year


In [84]:
link_df

,BBRefName,BBRefLink,BBRefID,BBRefBirthDate,NBAName,NBALink,NBAID,NBABirthDate,ESPNName,ESPNLink,ESPNID,ESPNBirthDate,SpotracName,SpotracLink,SpotracID
0,A.J. Hammons,https://www.basketball-reference.com/players/h...,hammoaj01,8/27/1992,AJ Hammons,https://stats.nba.com/player/1627773/,1627773,8/27/1992,AJ Hammons,http://www.espn.com/nba/player/_/id/2991178/aj...,2991178.0,8/27/1992,A.J. Hammons,https://www.spotrac.com/redirect/player/20252/,20252.0
1,A.J. Price,https://www.basketball-reference.com/players/p...,priceaj01,10/7/1986,AJ Price,https://stats.nba.com/player/201985/,201985,10/7/1986,A.J. Price,http://www.espn.com/nba/player/_/id/4010/aj-price,4010.0,10/7/1986,A.J. Price,https://www.spotrac.com/redirect/player/6292/,6292.0
2,Aaron Brooks,https://www.basketball-reference.com/players/b...,brookaa01,1/14/1985,Aaron Brooks,https://stats.nba.com/player/201166/,201166,1/14/1985,Aaron Brooks,http://www.espn.com/nba/player/_/id/3192/aaron...,3192.0,1/14/1985,Aaron Brooks,https://www.spotrac.com/redirect/player/2390/,2390.0
3,Aaron Gordon,https://www.basketball-reference.com/players/g...,gordoaa01,9/16/1995,Aaron Gordon,https://stats.nba.com/player/203932/,203932,9/16/1995,Aaron Gordon,http://www.espn.com/nba/player/_/id/3064290/aa...,3064290.0,9/16/1995,Aaron Gordon,https://www.spotrac.com/redirect/player/15356/,15356.0
4,Aaron Gray,https://www.basketball-reference.com/players/g...,grayaa01,12/7/1984,Aaron Gray,https://stats.nba.com/player/201189/,201189,12/7/1984,Aaron Gray,http://www.espn.com/nba/player/_/id/3207/aaron...,3207.0,12/7/1984,Aaron Gray,https://www.spotrac.com/redirect/player/2244/,2244.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4524,NaN,NaN,NaN,NaN,Terrico White,https://stats.nba.com/player/202358/,202358,3/7/1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4525,NaN,NaN,NaN,NaN,Lindell Wigginton,https://stats.nba.com/player/1629623/,1629623,3/28/1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4526,NaN,NaN,NaN,NaN,Ward Williams,https://stats.nba.com/player/78573/,78573,6/26/1923,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4527,NaN,NaN,NaN,NaN,Hank Zeller,https://stats.nba.com/player/78642/,78642,7/10/1918,NaN,NaN,NaN,NaN,NaN,NaN,NaN
